# CONCAT _0, _1 zarr to have only on file per type

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj
from rasterio.transform import Affine

import matplotlib.pyplot as plt
import matplotlib.colors as cl

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.geodesic as cgeo

crs = ccrs.PlateCarree()
import cmocean.cm as cm

from xgcm import Grid
from xhistogram.xarray import histogram
import warnings

warnings.filterwarnings("ignore")

import os
from glob import glob

import histlib.box as box
import histlib.aviso as aviso
import histlib.cstes as cstes
from histlib.cstes import labels, zarr_dir

/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


KeyboardInterrupt: 

In [2]:
zarr_dir_ok = zarr_dir + "_ok"

- i=0,1 :  jobs=6, cores = 26 
- i=2,3 :  jobs=1, cores = 26 
- i=4 :    jobs=1, cores = 10
- i=5 :    jobs=1, cores = 26
- i=6 :    jobs=4, cores = 26

In [3]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster

    # cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    # cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=10, processes=10, walltime="01:00:00")
    w = cluster.scale(jobs=3)
else:
    from dask.distributed import Client, LocalCluster

    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.56:8787/status,
Dashboard: http://10.148.1.56:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.56:59368,Workers: 0
Dashboard: http://10.148.1.56:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# AVISO

In [4]:
for l in ["gps_Jason-3_2020"]:
    files = sorted(glob(os.path.join(zarr_dir, f"aviso_{l}_*.zarr")))
    for f in files:
        ds = xr.open_dataset(f).chunk({"obs": 500})
        if (
            ds.dims["aviso_time"] == 25
        ):  # keep only five days of aviso (error for first drifter type, corrected after because too long)
            ds = ds.isel(aviso_time=slice(10, 15))
        if "_0." in f:  # overwrite
            ds.to_zarr(os.path.join(zarr_dir_ok, "aviso", f"aviso_{l}.zarr"), mode="w")
        else:  # append_dim
            ds.to_zarr(
                os.path.join(zarr_dir_ok, "aviso", f"aviso_{l}.zarr"), append_dim="obs"
            )
    print(l)

gps_Jason-3_2020


# ERASTAR

In [4]:
for l in labels:
    files = sorted(glob(os.path.join(zarr_dir, f"erastar_{l}_*.zarr")))
    for f in files:
        ds = xr.open_dataset(f).chunk({"obs": 500})
        if "_0." in f:  # overwrite
            ds.to_zarr(
                os.path.join(zarr_dir_ok, "erastar", f"erastar_{l}.zarr"), mode="w"
            )
        else:  # append_dim
            ds.to_zarr(
                os.path.join(zarr_dir_ok, "erastar", f"erastar_{l}.zarr"),
                append_dim="obs",
            )
    print(l)

gps_Jason-3_2020
argos_Jason-3_2020
gps_SARAL_2020
argos_SARAL_2020
gps_Cryosat-2_2020
argos_Cryosat-2_2020
gps_Sentinel-3_A_2020
argos_Sentinel-3_A_2020
gps_Sentinel-3_B_2020
argos_Sentinel-3_B_2020
gps_Jason-3_2019
argos_Jason-3_2019
gps_SARAL_2019
argos_SARAL_2019
gps_Cryosat-2_2019
argos_Cryosat-2_2019
gps_Sentinel-3_A_2019
argos_Sentinel-3_A_2019
gps_Sentinel-3_B_2019
argos_Sentinel-3_B_2019
gps_Jason-3_2018
argos_Jason-3_2018
gps_SARAL_2018
argos_SARAL_2018
gps_Cryosat-2_2018
argos_Cryosat-2_2018
gps_Sentinel-3_A_2018
argos_Sentinel-3_A_2018
gps_Sentinel-3_B_2018
argos_Sentinel-3_B_2018
gps_Jason-3_2017
argos_Jason-3_2017
gps_Jason-2_2017
argos_Jason-2_2017
gps_SARAL_2017
argos_SARAL_2017
gps_Cryosat-2_2017
argos_Cryosat-2_2017
gps_Sentinel-3_A_2017
argos_Sentinel-3_A_2017
gps_Jason-3_2016
argos_Jason-3_2016
gps_Jason-2_2016
argos_Jason-2_2016
gps_SARAL_2016
argos_SARAL_2016
gps_Cryosat-2_2016
argos_Cryosat-2_2016
gps_Sentinel-3_A_2016
argos_Sentinel-3_A_2016
gps_Jason-2_2015
arg

# MATCHUP

In [ ]:
for l in labels:
    f = os.path.join(zarr_dir_ok, "matchup", f"matchup_{l}.zarr")
    ds = xr.open_dataset(f).drop("drifter_typebuoy").dropna("obs").chunk({"obs": 500})
    if l == labels[0]:  # overwrite
        ds.to_zarr(os.path.join(zarr_dir_ok, "matchup.zarr"), mode="w")
    else:  # append_dim
        ds.to_zarr(os.path.join(zarr_dir_ok, "matchup.zarr"), append_dim="obs")
    print(l)

gps_Jason-3_2020
argos_Jason-3_2020
gps_SARAL_2020
argos_SARAL_2020
gps_Cryosat-2_2020
argos_Cryosat-2_2020
gps_Sentinel-3_A_2020
argos_Sentinel-3_A_2020
gps_Sentinel-3_B_2020
argos_Sentinel-3_B_2020
gps_Jason-3_2019
argos_Jason-3_2019
gps_SARAL_2019
argos_SARAL_2019
gps_Cryosat-2_2019
argos_Cryosat-2_2019
gps_Sentinel-3_A_2019
argos_Sentinel-3_A_2019
gps_Sentinel-3_B_2019
argos_Sentinel-3_B_2019
gps_Jason-3_2018
argos_Jason-3_2018
gps_SARAL_2018
argos_SARAL_2018
gps_Cryosat-2_2018
argos_Cryosat-2_2018
gps_Sentinel-3_A_2018
argos_Sentinel-3_A_2018
gps_Sentinel-3_B_2018
argos_Sentinel-3_B_2018
gps_Jason-3_2017


In [6]:
ds = xr.open_dataset(os.path.join(zarr_dir_ok, "matchup.zarr")).chunk({"obs": 500})

In [8]:
cluster.close()

In [7]:
ds

<xarray.Dataset>
Dimensions:                           (obs: 4395310)
Coordinates:
    alti___distance                   (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    alti___time_difference            (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    lat                               (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    lon                               (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
  * obs                               (obs) <U42 'gps_Jason-3_2020__0' ... 'a...
    time                              (obs) datetime64[ns] dask.array<chunksize=(500,), meta=np.ndarray>
Data variables: (12/46)
    alti_ggx_adt_filtered             (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    alti_ggx_adt_unfiltered           (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    alti_ggx_adt_unfiltered_denoised  (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    alti_ggx_dac                      (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    alti_ggx_internal_tide            (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    alti_ggx_mdt                      (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    ...                                ...
    es_cstrio_z0_drifter_wd_x         (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    es_cstrio_z0_drifter_wd_y         (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    es_cstrio_z15_alti_wd_x           (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    es_cstrio_z15_alti_wd_y           (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    es_cstrio_z15_drifter_wd_x        (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    es_cstrio_z15_drifter_wd_y        (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
Attributes:
    __id:                   CMEMS-L3-SLA-Cryosat-2__argos
    __time_coverage_end:    2011-01-01T00:00:00
    __time_coverage_start:  2010-07-01T00:00:00